# Approx Lab

The goal of this lab is to gain experience using approximation algorithms.

For the purpose of this assignment, you will be implementing the knapsack problem. Before continuing, the following cell contains any dependencies you may need for this assignment.

In [1]:
import pandas as pd

## Background Info

### Knapsack Problem
We are actually going to be looking at a specific (hopefully fun) implementation of the knapsack problem.

The traditional knapsack problem asks you to pack a set of items, with given weights and values into a "knapsack" with a maximum capacity. You cannot pack more items than the capacity of the knapsack. The goal is to select a subset of items that maximize the total value with what can fit in the knapsack. The name "knapsack" comes from the similarity to a hiker trying to pack a bag without going over a certain weight.

The knapsack problem is an NP-Hard optimization problem, which is why it is one of the classic approximation algorithms.

### Specifications for this Lab
The variation that we will be doing in this assignment is referred to as an 0/1 knapsack problem, meaning that items must be completely put in the knapsack or they must be left behind (you can't have a fractional portion of an item). We will be representing this as a simple fantasy sports league - specifically Formula 1 drivers. 

You are provided with a list of F1 drivers and constructors with given costs and points as well as a budget of $100M. You will be selecting five drivers and two constructors (teams). 

### Scoring Information
Points are gained from qualification results for drivers and constructors as well as race results for drivers and constructors. 

<!-- Qualifications
- Drivers score based on their resulting position 
- Constructor score the combined total of their two drivers in qualifying plus a modifier based on how far both their drivers get. -->
<!-- 
| Drivers    | Points |
| -------- | ------- |
| Pole Position (1st Place) | 10 |
| 2nd Place | 9 |
| 3rd Place | 8 |
| 4th Place | 7 |
| 5th Place | 6 |
| 6th Place | 5 |
| 7th Place | 4 |
| 8th Place | 3 |
| 9th Place | 2 |
| 10th Place | 1 |
| 11th-20th Place | 0 |
| No Time Set | 0 |
| Disqualification | 0 | -->


<!-- | Constructors | Points |
| ----------- | ------- |
| Neither Driver Reaches Q2 | -1 |
| One Driver Reaches Q2 | 1 |
| Both Drivers Reach Q2 | 5 |
| Both Drivers Reach Q3 | 10 | -->


<!-- Race
- Drivers score based on their resulting position, position difference from starting order, and some additional bonuses
- Constructors score a combined total of their two drivers' results (excluding the Driver of the Day bonus) ~~plus a modifier based on pitstop times~~ -->

<!-- | Drivers | Points |
| -------- | ------- |
| Positions Gained | 1/position |
| Positions Lost | -1/position |
| Overtakes Made | 1/overtake |
| Fastest Lap | 10 |
| Driver of the Day | 10 | -->

<!-- | Race Result | Points |
| -------- | ------- |
| 1st Place | 25 |
| 2nd Place | 18 |
| 3rd Place | 15 |
| 4th Place | 12 |
| 5th Place | 10 |
| 6th Place | 8 |
| 7th Place | 6 |
| 8th Place | 4 |
| 9th Place | 2 |
| 10th Place | 1 |
| 11th-20th Place | 0 |
| DNF | -20 |
| Disqualification | -20 | -->

Fortunately for you, we have access to the current point standings post-2025 China GP for drivers & constructors, which are available in `drivers.csv` & `constructors.csv`.

*Note: The costs are in millions, so 30 is actually 30 M.

### Questions (Answered in Various Parts of the Lab): 
1. What is the general setup of the problem?
2. Determining the mathematical functions of the problem
3. Implementing a greedy algorithm


## A. Assessing the Problem

The values that are provided for this lab are from the points distribution PRIOR to the Japan Grand Prix (which means we can actually compare with the race results at the submission deadline!)

Create a fantasy team https://fantasy.formula1.com/en/create-team for the upcoming Grand Prix to get a better idea of what drivers, constructors, points, and the budget are. Who is on your team & how much did you spend? Don't spend too much time on this, it really is just to get familiar with the format.

For this exercise, I would create a team focusing on high value-to-cost ratio players. A sample team might include:

Drivers: Lando Norris, Nico Hulkenberg, Lance Stroll, Esteban Ocon, Oliver Bearman
Constructors: Haas, Mercedes
Total budget spent: ~$92.8M out of $100M

What represents the "items" in the knapsack? 

The "items" in this knapsack problem are the individual F1 drivers and constructors that can be selected for the fantasy team.

What represents the weights?

The "weights" in this problem are the costs (in millions) associated with each driver and constructor. These costs represent how much of the budget is consumed by selecting each item, similar to how weights take up capacity in a traditional knapsack problem.

What represents the values?

The "values" in this problem are the points that each driver and constructor has accumulated. These points represent the benefit gained by selecting each item, which is what we're trying to maximize while staying within our budget constraint.
The goal is to select 5 drivers and 2 constructors (specific quantity constraints) while maximizing total points and keeping total cost under $100M (the knapsack capacity constraint).

In [2]:
# Load the data from the CSV files into a format that you are comfortable working with
drivers_df = pd.read_csv('drivers.csv')
constructors_df = pd.read_csv('constructors.csv')

# Print/Display your data to confirm successful loading
print("Drivers Data:")
print(drivers_df)
print("\nConstructors Data:")
print(constructors_df)

Drivers Data:
                   Driver  Points  Cost
0            Lando Norris     100  29.6
1   Andrea Kimi Antonelli      61  19.3
2          George Russell      60  21.4
3          Max Verstappen      59  28.6
4           Oscar Piastri      55  23.0
5            Lance Stroll      33   9.3
6            Esteban Ocon      32   8.1
7          Alexaner Albon      28  12.8
8         Nico Hulkenberg      22   7.6
9          Oliver Bearman      22   6.7
10         Lewis Hamilton       3  23.6
11           Yuki Tsunoda       0  16.8
12            Liam Lawson       0   8.4
13        Charles LeClerc       0  25.3
14           Pierre Gasly      -7  10.6
15           Isack Hadjar      -9   5.0
16            Jack Doohan      -9   6.0
17           Carlos Sainz     -11  11.9
18      Gabriel Bortoleto     -11   4.8
19        Fernando Alonso     -36   7.6

Constructors Data:
       Constructor  Points  Cost
0          McLaren     172  30.6
1         Mercedes     136  23.3
2  Red Bull Racing      86 

# B. Integer Program

Before implementing our approx algorithm, we need to first consider the integer program. 

Given the following variables, create an integer program that maximizes the total points of drivers while remaining under the budget: 
- $x_i$ is the decision variable that determines whether driver $i$ is included in the team or not
- $p_i$ is the # of points that driver $i$ is worth
- $c_i$ is the cost of driver $i$

For now, we will focus on just the drivers and ignore the constructors. We will also temporarily reduce our budget to **$50M**

While we are not planning on implementing the full integer program, it helps us quantify parts of the problem

## Decision Variable & Constraints

We can make helper functions for our future algorithm based on constraints that we derive when forming the IP: 
1. Which drivers should be included in our solution? 
2. What is our maximum cost?
3. What is our maximum # of drivers? 

In [3]:
budget = 50 # in millions

# Helper functions for our constraints
def is_within_budget(selected_items, budget):
    """Check if the total cost of selected items is within budget"""
    total_cost = sum(item['Cost'] for item in selected_items)
    return total_cost <= budget

def is_max_drivers_reached(selected_drivers, max_drivers=5):
    """Check if we've reached the maximum number of drivers"""
    return len(selected_drivers) >= max_drivers

# C. Greedy Algorithm


There are multiple ways we can go about solving the knapsack problem. For this lab, we will be focusing on a greedy algorithm and a variant of that.

Let's start with a basic greedy algorithm:
1. Calculate the value-to-cost ratio for each driver 
2. Sort drivers in non-increasing order of value-to-cost
2. Greedily select drivers from list until either budget or max drivers constraint is hit

Use the helper methods from the previous part in your implementation.

In [4]:
def value_to_cost(items):
    """Calculate the value-to-cost ratio for each item"""
    # Create a copy of the items with an additional value-to-cost ratio
    items_with_ratio = []
    for item in items:
        # Create a copy of the item
        item_with_ratio = item.copy()
        # Add the value-to-cost ratio
        item_with_ratio['Ratio'] = item['Points'] / item['Cost']
        items_with_ratio.append(item_with_ratio)
    return items_with_ratio

def sort_items(items):
    """Sort items in non-increasing order of value-to-cost ratio"""
    return sorted(items, key=lambda x: x['Ratio'], reverse=True)

def greedy_driver_select(drivers, budget=50, max_drivers=5):
    """
    Basic greedy algorithm to select drivers based on value-to-cost ratio
    """
    # Calculate value-to-cost ratio
    drivers_with_ratio = value_to_cost(drivers)
    
    # Sort by value-to-cost ratio
    sorted_drivers = sort_items(drivers_with_ratio)
    
    selected_drivers = []
    remaining_budget = budget
    
    # Greedily select drivers
    for driver in sorted_drivers:
        if len(selected_drivers) < max_drivers and driver['Cost'] <= remaining_budget:
            selected_drivers.append(driver)
            remaining_budget -= driver['Cost']
    
    # Calculate total points and cost
    total_points = sum(driver['Points'] for driver in selected_drivers)
    total_cost = sum(driver['Cost'] for driver in selected_drivers)
    
    return {
        'selected_drivers': selected_drivers,
        'total_points': total_points,
        'total_cost': total_cost,
        'remaining_budget': remaining_budget
    }

# Convert DataFrame to list of dictionaries for easier processing
drivers_list = drivers_df.to_dict('records')

# Run the basic greedy algorithm
basic_greedy_result = greedy_driver_select(drivers_list)
print("\nBasic Greedy Algorithm Results:")
print("Selected Drivers:")
for driver in basic_greedy_result['selected_drivers']:
    print(f"  {driver['Driver']}: Points = {driver['Points']}, Cost = {driver['Cost']}")
print(f"Total Points: {basic_greedy_result['total_points']}")
print(f"Total Cost: {basic_greedy_result['total_cost']}")
print(f"Remaining Budget: {basic_greedy_result['remaining_budget']}")


Basic Greedy Algorithm Results:
Selected Drivers:
  Esteban Ocon: Points = 32, Cost = 8.1
  Lance Stroll: Points = 33, Cost = 9.3
  Lando Norris: Points = 100, Cost = 29.6
Total Points: 165
Total Cost: 47.0
Remaining Budget: 2.999999999999993


Try running your basic greedy algorithm. 

You may notice a few flaws:
1. The greedy algorithm is only looking at value-to-cost ratio, which may miss out on an overall improvement in value that still fits within the budget
2. Depending on your implementation, you may not currently be meeting the "5 total drivers" constraint - this is fine


## Redux Greedy Algorithm
We are going to next implement a small adjustment to this algorithm in order to improve its performance. \
This is a 2-approximation for the knapsack problem. 

1. Calculate the value-to-cost ratio for each driver
2. Sort drivers in non-increasing order of value-to-cost
3. Greedily add items until we hit an item that is too big
4. Compare the too big item against the already existing list of items & pick the better of the two

It might help to make a separate function to handle #4. 

In [5]:
def redux_greedy_driver_select(drivers, budget=50, max_drivers=5):
    """
    2-approximation algorithm for the knapsack problem
    """
    # Calculate value-to-cost ratio
    drivers_with_ratio = value_to_cost(drivers)
    
    # Sort by value-to-cost ratio
    sorted_drivers = sort_items(drivers_with_ratio)
    
    # Method 1: Greedily add items until we hit an item that is too big
    selected_drivers_1 = []
    remaining_budget_1 = budget
    
    for driver in sorted_drivers:
        if len(selected_drivers_1) < max_drivers and driver['Cost'] <= remaining_budget_1:
            selected_drivers_1.append(driver)
            remaining_budget_1 -= driver['Cost']
    
    # Calculate total points for method 1
    total_points_1 = sum(driver['Points'] for driver in selected_drivers_1)
    
    # Method 2: Find the most valuable single item
    max_value_driver = max(drivers, key=lambda x: x['Points'])
    
    # Compare the results of both methods
    if max_value_driver['Points'] > total_points_1 and max_value_driver['Cost'] <= budget:
        return {
            'selected_drivers': [max_value_driver],
            'total_points': max_value_driver['Points'],
            'total_cost': max_value_driver['Cost'],
            'remaining_budget': budget - max_value_driver['Cost']
        }
    else:
        return {
            'selected_drivers': selected_drivers_1,
            'total_points': total_points_1,
            'total_cost': budget - remaining_budget_1,
            'remaining_budget': remaining_budget_1
        }

# Run the redux greedy algorithm
redux_greedy_result = redux_greedy_driver_select(drivers_list)
print("\nRedux Greedy Algorithm Results:")
print("Selected Drivers:")
for driver in redux_greedy_result['selected_drivers']:
    print(f"  {driver['Driver']}: Points = {driver['Points']}, Cost = {driver['Cost']}")
print(f"Total Points: {redux_greedy_result['total_points']}")
print(f"Total Cost: {redux_greedy_result['total_cost']}")
print(f"Remaining Budget: {redux_greedy_result['remaining_budget']}")


Redux Greedy Algorithm Results:
Selected Drivers:
  Esteban Ocon: Points = 32, Cost = 8.1
  Lance Stroll: Points = 33, Cost = 9.3
  Lando Norris: Points = 100, Cost = 29.6
Total Points: 165
Total Cost: 47.00000000000001
Remaining Budget: 2.999999999999993


# D. Reintroducing Constructors

We've been neglecting our constructors so far and cutting our budget. Let's make them feel included in our processes again. 

## Integer Program

We should start by redefining our original IP to see what constraints we need to add. \
We've added some more variables to our IP: 
- $x_j$ is the decision variable that determines whether constructor $j$ is included in the team or not 
- $p_j$ is the # of points that constructor $j$ is worth
- $c_j$ is the cost of constructor $j$

Our budget has also been increased back to **100M**. 

## Constraints 

To redefine our greedy algorithm to include constructors, we have to also write helpers that correspond to our new constraints.

In [6]:
budget = 100 # in millions

def is_max_constructors_reached(selected_constructors, max_constructors=2):
    return len(selected_constructors) >= max_constructors

## Greedy Algorithm

Update the greedy algorithm to calculate for buying both constructors and drivers. \
Remember, your list of non-increasing value-to-cost ratios should be a combination of both drivers & constructors. 

In [7]:
def greedy_f1(drivers, constructors, budget=100, max_drivers=5, max_constructors=2):
    # Convert to list if DataFrames are provided
    if isinstance(drivers, pd.DataFrame):
        drivers = drivers.to_dict('records')
    if isinstance(constructors, pd.DataFrame):
        constructors = constructors.to_dict('records')
    
    # Calculate value-to-cost ratio for both drivers and constructors
    drivers_with_ratio = value_to_cost(drivers)
    constructors_with_ratio = value_to_cost(constructors)
    
    # Add a type field to distinguish between drivers and constructors
    for driver in drivers_with_ratio:
        driver['Type'] = 'Driver'
    for constructor in constructors_with_ratio:
        constructor['Type'] = 'Constructor'
    
    # Combine both lists
    all_items = drivers_with_ratio + constructors_with_ratio
    
    # Sort by value-to-cost ratio
    sorted_items = sort_items(all_items)
    
    selected_drivers = []
    selected_constructors = []
    remaining_budget = budget
    
    # Greedily select items
    for item in sorted_items:
        if item['Type'] == 'Driver' and not is_max_drivers_reached(selected_drivers, max_drivers) and item['Cost'] <= remaining_budget:
            selected_drivers.append(item)
            remaining_budget -= item['Cost']
        elif item['Type'] == 'Constructor' and not is_max_constructors_reached(selected_constructors, max_constructors) and item['Cost'] <= remaining_budget:
            selected_constructors.append(item)
            remaining_budget -= item['Cost']
    
    # Calculate total points and cost
    total_drivers_points = sum(driver['Points'] for driver in selected_drivers)
    total_constructors_points = sum(constructor['Points'] for constructor in selected_constructors)
    total_points = total_drivers_points + total_constructors_points
    total_cost = sum(item['Cost'] for item in selected_drivers + selected_constructors)
    
    return {
        'selected_drivers': selected_drivers,
        'selected_constructors': selected_constructors,
        'total_drivers_points': total_drivers_points,
        'total_constructors_points': total_constructors_points,
        'total_points': total_points,
        'total_cost': total_cost,
        'remaining_budget': remaining_budget
    }

# Convert DataFrames to list of dictionaries
constructors_list = constructors_df.to_dict('records')

# Run the combined greedy algorithm
combined_greedy_result = greedy_f1(drivers_list, constructors_list)
print("\nCombined Greedy Algorithm Results:")
print("Selected Drivers:")
for driver in combined_greedy_result['selected_drivers']:
    print(f"  {driver['Driver']}: Points = {driver['Points']}, Cost = {driver['Cost']}")
print("\nSelected Constructors:")
for constructor in combined_greedy_result['selected_constructors']:
    print(f"  {constructor['Constructor']}: Points = {constructor['Points']}, Cost = {constructor['Cost']}")
print(f"Total Drivers Points: {combined_greedy_result['total_drivers_points']}")
print(f"Total Constructors Points: {combined_greedy_result['total_constructors_points']}")
print(f"Total Points: {combined_greedy_result['total_points']}")
print(f"Total Cost: {combined_greedy_result['total_cost']}")
print(f"Remaining Budget: {combined_greedy_result['remaining_budget']}")

# Implementing the 2-approximation algorithm for the combined case
def redux_greedy_f1(drivers, constructors, budget=100, max_drivers=5, max_constructors=2):
    """
    2-approximation algorithm for the combined knapsack problem
    """
    # Convert to list if DataFrames are provided
    if isinstance(drivers, pd.DataFrame):
        drivers = drivers.to_dict('records')
    if isinstance(constructors, pd.DataFrame):
        constructors = constructors.to_dict('records')
    
    # Method 1: Regular greedy
    regular_greedy_result = greedy_f1(drivers, constructors, budget, max_drivers, max_constructors)
    
    # Method 2: Find the most valuable combination of exactly 1 driver and 1 constructor
    best_combo = None
    best_combo_points = 0
    
    for driver in drivers:
        for constructor in constructors:
            combo_cost = driver['Cost'] + constructor['Cost']
            combo_points = driver['Points'] + constructor['Points']
            
            if combo_cost <= budget and combo_points > best_combo_points:
                best_combo = {
                    'driver': driver,
                    'constructor': constructor,
                    'points': combo_points,
                    'cost': combo_cost
                }
                best_combo_points = combo_points
    
    # Compare the results
    if best_combo and best_combo['points'] > regular_greedy_result['total_points']:
        return {
            'selected_drivers': [best_combo['driver']],
            'selected_constructors': [best_combo['constructor']],
            'total_drivers_points': best_combo['driver']['Points'],
            'total_constructors_points': best_combo['constructor']['Points'],
            'total_points': best_combo['points'],
            'total_cost': best_combo['cost'],
            'remaining_budget': budget - best_combo['cost']
        }
    else:
        return regular_greedy_result

# Run the redux combined greedy algorithm
redux_combined_result = redux_greedy_f1(drivers_list, constructors_list)
print("\nRedux Combined Greedy Algorithm Results:")
print("Selected Drivers:")
for driver in redux_combined_result['selected_drivers']:
    print(f"  {driver['Driver']}: Points = {driver['Points']}, Cost = {driver['Cost']}")
print("\nSelected Constructors:")
for constructor in redux_combined_result['selected_constructors']:
    print(f"  {constructor['Constructor']}: Points = {constructor['Points']}, Cost = {constructor['Cost']}")
print(f"Total Drivers Points: {redux_combined_result['total_drivers_points']}")
print(f"Total Constructors Points: {redux_combined_result['total_constructors_points']}")
print(f"Total Points: {redux_combined_result['total_points']}")
print(f"Total Cost: {redux_combined_result['total_cost']}")
print(f"Remaining Budget: {redux_combined_result['remaining_budget']}")

# Bonus: Implement a function to evaluate if we've met all constraints
def check_constraints(result, budget=100, required_drivers=5, required_constructors=2):
    """Check if all constraints have been met"""
    num_drivers = len(result['selected_drivers'])
    num_constructors = len(result['selected_constructors'])
    total_cost = result['total_cost']
    
    constraints_met = True
    issues = []
    
    if num_drivers != required_drivers:
        constraints_met = False
        issues.append(f"Expected {required_drivers} drivers, but selected {num_drivers}")
    
    if num_constructors != required_constructors:
        constraints_met = False
        issues.append(f"Expected {required_constructors} constructors, but selected {num_constructors}")
    
    if total_cost > budget:
        constraints_met = False
        issues.append(f"Budget exceeded: {total_cost} > {budget}")
    
    return {
        'constraints_met': constraints_met,
        'issues': issues
    }

# Check if our combined greedy solution meets all constraints
constraint_check = check_constraints(combined_greedy_result)
print("\nConstraint Check:")
print(f"All constraints met: {constraint_check['constraints_met']}")
if not constraint_check['constraints_met']:
    print("Issues:")
    for issue in constraint_check['issues']:
        print(f"  - {issue}")


Combined Greedy Algorithm Results:
Selected Drivers:
  Esteban Ocon: Points = 32, Cost = 8.1
  Lance Stroll: Points = 33, Cost = 9.3
  Lando Norris: Points = 100, Cost = 29.6
  Oliver Bearman: Points = 22, Cost = 6.7
  Nico Hulkenberg: Points = 22, Cost = 7.6

Selected Constructors:
  Haas: Points = 61, Cost = 8.2
  Mercedes: Points = 136, Cost = 23.3
Total Drivers Points: 209
Total Constructors Points: 197
Total Points: 406
Total Cost: 92.8
Remaining Budget: 7.199999999999994

Redux Combined Greedy Algorithm Results:
Selected Drivers:
  Esteban Ocon: Points = 32, Cost = 8.1
  Lance Stroll: Points = 33, Cost = 9.3
  Lando Norris: Points = 100, Cost = 29.6
  Oliver Bearman: Points = 22, Cost = 6.7
  Nico Hulkenberg: Points = 22, Cost = 7.6

Selected Constructors:
  Haas: Points = 61, Cost = 8.2
  Mercedes: Points = 136, Cost = 23.3
Total Drivers Points: 209
Total Constructors Points: 197
Total Points: 406
Total Cost: 92.8
Remaining Budget: 7.199999999999994

Constraint Check:
All const